#### Making Market Timming

In [1]:
def making_mt(start,end,code):
    from pykrx import stock
    df = stock.get_index_fundamental(start,end,code)
    df = df.drop(['등락률','PER','선행PER','PBR','배당수익률'],axis=1)
    
    ''' code : KOSPI ('1001') KOSDAQ ('2001') '''
    if code=='1001': 
        df['sma20'] = df['종가'].rolling(20).mean()
        df['mt'] = df['종가']>df['sma20']
        df = df.dropna()
        return df
    elif code=='2001':     
        df['sma3'] = df['종가'].rolling(3).mean()
        df['sma5'] = df['종가'].rolling(5).mean()
        df['sma10'] = df['종가'].rolling(10).mean()
        df['mt'] = ((df['종가']>df['sma3'])|(df['종가']>df['sma5'])|(df['종가']>df['sma10']))
        df = df.dropna()
        return df
    else:
        print('올바른 종목 code 입력')

In [2]:
def making_mt_plot(df,code):
    import matplotlib.pyplot as plt
    from matplotlib import dates
    if code=='1001': 
        plt.figure(figsize = (50, 10))
        plt.title("Kospi Market timming")
        plt.plot(df["종가"])
        plt.plot(df['sma20'])
        plt.scatter(df.loc[df['mt']==1].index, df['종가'][df['mt']==1],
                    color = 'red',
                    label='Go',
                    marker ='^',
                    alpha=0.8)
        plt.scatter(df.loc[df['mt']==0].index, df['종가'][df['mt']==0],
                    color = 'blue',
                    label='Stop',
                    marker ='v',
                    alpha=0.8)
        plt.xticks(rotation=45)
        plt.legend()
        plt.show()
    elif code=='2001':
        plt.figure(figsize = (50, 10))
        plt.title("Kosdaq Market timming")
        plt.plot(df["종가"])
        plt.plot(df['sma3'])
        plt.plot(df['sma5'])
        plt.plot(df['sma10'])
        plt.scatter(df.loc[df['mt']==1].index, df['종가'][df['mt']==1],
                    color = 'red',
                    label='Go',
                    marker ='^',
                    alpha=0.8)
        plt.scatter(df.loc[df['mt']==0].index, df['종가'][df['mt']==0],
                    color = 'blue',
                    label='Stop',
                    marker ='v',
                    alpha=0.8)
        plt.xticks(rotation=45)
        plt.legend()
        plt.show()
    else:
        print('Error : Code check')
    

## 설명 
* backtest 시작 , 끝날짜를 for 문으로 돌린다 ex) for i in range(start,end)  
* 시작 날짜에 maket Timming 이 true면 아래전략 시행  
* sort_vol = 시작 날짜 누적거래대금 범위에 속하는 종목 df로만든다.  output : df_vol
* search momentum signal = df_vol과 df_raw 를 활용하여 Momentum 값을 계산하고 signal을 만든다 
* True 인 종목을 매수한다. 현금계좌잔고 감소 , 보유잔고 증가 (주가*수량)  

<익일>  
* 다음날 보유잔고에 매도 조건을 확인한다 매도조건이 True이면 매도한다. 현금계좌잔고 증가 (당일 종가 * 수량) 보유잔고 감소  
* 매도 후 전일과 같이 위 과정을 반복  

<최종>  
* 수익률 출력  
* MDD 출력 (이전략으로 매매를 했을때 최대 몇% 까지 손실을 보았었는가)  
* 샤프지수출력 (이 전략이아니라 그냥 지수에 투자했을 때에 비해 변동성이 얼마나 큰가 )
* 승률 (이전략을 했을때 + 수익을 본횟수 / 전체횟수)  
* log (활용 무궁무진 : 매수 매도 기록을 전부기록 추후 확률 통계에 기반이됨)  

<고급>
* momentum 지수가 몇일 때 매수를 하면 다음날 상승확률이 가장 높은지 통계적검증 해서 파라미터 조정가능 


#### Making 거래대금조회


In [ ]:
def sort_vol(df,date,range):
    df = df[(df['누적거래대금']>=range)&(df['날짜']==date)]
    return df

#### Momentum Search

In [25]:
def search_momentum_signal(df_vol, df_raw):
    li = df_vol['종목명'].values
    dt = df_vol['날짜'].values[0]
    df_vol['Buy'] = None
    df_vol['momentum'] = None
    for i in li:
        today_idx = df_raw[(df_raw['종목명']==i)&(df_raw['날짜']==dt)].index
        day5_idx = df_raw[(df_raw['종목명']==i)&(df_raw['날짜']==dt)].index-5
        momentum = df_raw.loc[today_idx]['종가'].values/df_raw.loc[day5_idx]['종가'].values
        df_vol.loc[df_vol['종목명']==i,'momentum']=momentum[0]
        df_vol.loc[df_vol['종목명']==i,'Buy']=df_vol['momentum']>=1
    return df_vol
        

In [31]:
df_vol

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명
990,20211220,156000,157700,162900,154900,317752,50211593600,LX세미콘
2615,20211220,58400,56600,60100,55900,2704260,158734327800,SK스퀘어
3182,20211220,172500,175500,176000,171000,246385,42610001000,카카오페이
3486,20211220,33400,34950,35500,33100,644051,21809554050,케이카
3803,20211220,91900,92100,93400,91600,462510,42754787200,현대중공업
...,...,...,...,...,...,...,...,...
1590459,20211220,14000,15350,15700,13650,1141819,16518178500,신원종합개발
1600171,20211220,11800,13350,13700,11800,1100617,13808207000,플레이위드
1603813,20211220,1990,1930,2240,1905,5661042,11978889445,모베이스전자
1618381,20211220,3930,4015,4415,3930,3033369,12652994885,바른손


In [32]:
search_momentum_signal(df_vol, df_raw)

C:\Users\mi-bi\AppData\Local\Temp\ipykernel_19536\838232746.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vol['Buy'] = None
C:\Users\mi-bi\AppData\Local\Temp\ipykernel_19536\838232746.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vol['momentum'] = None


,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,Buy,momentum
990,20211220,156000,157700,162900,154900,317752,50211593600,LX세미콘,True,1.029703
2615,20211220,58400,56600,60100,55900,2704260,158734327800,SK스퀘어,False,0.926984
3182,20211220,172500,175500,176000,171000,246385,42610001000,카카오페이,False,0.907895
3486,20211220,33400,34950,35500,33100,644051,21809554050,케이카,False,0.835
3803,20211220,91900,92100,93400,91600,462510,42754787200,현대중공업,False,0.866981
...,...,...,...,...,...,...,...,...,...,...
1590459,20211220,14000,15350,15700,13650,1141819,16518178500,신원종합개발,False,0.795455
1600171,20211220,11800,13350,13700,11800,1100617,13808207000,플레이위드,False,0.861314
1603813,20211220,1990,1930,2240,1905,5661042,11978889445,모베이스전자,False,0.987593
1618381,20211220,3930,4015,4415,3930,3033369,12652994885,바른손,False,0.991173


In [3]:
import pandas as pd 

In [9]:
df_1 = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/data Row/5years_kospi.csv', index_col=0)
df_2 = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/data Row/5years_kosdaq.csv', index_col=0)
df_raw = pd.concat([df_1,df_2])

In [16]:
def sort_vol(df,date,range):
    df = df[(df['누적거래대금']>=range)&(df['날짜']==date)]
    return df

In [30]:
df_vol = sort_vol(df_raw, 20211220, 10000000000)

In [1]:
def make_momentum(df):
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        df.loc[idx[0]:idx[-1],'momentum'] = df.loc[idx[0]:idx[-1],'종가']/df.loc[idx[0]:idx[-1],'종가'].shift(5)*100